# Tutorial: Building Quantum Hamiltonians with the `quantum_simulation` Toolkit

Welcome to this tutorial\! The goal of this guide is to walk you through the process of constructing Hamiltonian matrices for various quantum systems using the powerful and flexible `quantum_simulation` toolkit.

The library is built on a few core principles:

  * **Modularity:** Operator definitions, the Hamiltonian builder, and pre-built physical models are separated into distinct modules (`operator`, `hamiltonian`, `models`).
  * **Package Structure:** All modules are part of a unified `quantum_simulation` package, allowing for clean and organized imports.
  * **Flexibility:** The `Hamiltonian` class acts as a universal builder, allowing you to add different types of terms (spin, boson, fermion) to construct the matrix for nearly any lattice model.
  * **Ease of Use:** For common systems, the `models` module provides ready-to-use classes that handle the construction for you and can even display a symbolic representation of the Hamiltonian.

In this tutorial, we will cover:

1.  **Setting up the `quantum_simulation` package** in Colab.
2.  **Understanding the core components** of the toolkit.
3.  **Building a spin model from scratch** (the Transverse Field Ising Model).
4.  **Building a fermionic model from scratch** (the Su-Schrieffer-Heeger Model).
5.  **Exploring the pre-built `models` module** and its catalog of Hamiltonians.
6.  **Using the `add_prebuilt_term`** for special cases.

## 1\. Setup and Installation

First, we need to structure our code as a Python package in the Colab environment and install the necessary dependencies.

In [1]:
try:
    import quantum_simulation as qs
    from quantum_simulation import operator as op
except ModuleNotFoundError:
    !git clone https://github.com/ToelUl/quantum-simulation.git
    !cp -r quantum-simulation/quantum_simulation ./
    import quantum_simulation as qs
    from quantum_simulation import operator as op

## 2\. The Core Components: An Overview

Let's briefly review the purpose of each main component.

  * **`operator.py`**: The foundation. It provides functions that generate the explicit matrix representations of quantum operators (e.g., Pauli matrices, fermionic operators).
  * **`hamiltonian.py`**: The heart of the toolkit. It contains the `Hamiltonian` class, which acts as the main builder for constructing the final matrix from individual terms.
  * **`models.py`**: A high-level, user-friendly interface containing pre-built classes for common physical models (e.g., `HeisenbergModel`).

Let's import these modules and look at a simple operator.

In [2]:
import torch

# Let's create a single Pauli-X operator for a spin-1/2 system
sx_matrix = op.pauli_x(spin=0.5)

print("Pauli-X matrix for spin=1/2:")
qs.show_matrix(sx_matrix)

Pauli-X matrix for spin=1/2:


Matrix([
[  0, 1.0],
[1.0,   0]])

## 3\. Building a Hamiltonian from Scratch: The 1D Transverse Field Ising Model (TFIM)

Now, we will build a Hamiltonian for a well-known model completely from scratch. The TFIM Hamiltonian is defined as:
$$H = -J \sum_{i=0}^{N-2} S^x_i S^x_{i+1} - h \sum_{i=0}^{N-1} S^z_i$$

This involves an interaction term ($J$) and a field term ($h$). Let's build it step-by-step.

In [3]:
# --- Step 1: Define model parameters ---
L = 4  # Number of sites in the chain
J = 1.0  # Interaction strength
h = 0.5  # Transverse field strength

# --- Step 2: Initialize the Hamiltonian builder ---
from quantum_simulation.hamiltonian import Hamiltonian
from quantum_simulation import operator as op
import quantum_simulation as qs

tfim_builder = Hamiltonian(lattice_length=L, spin=0.5)

print(f"Building a TFIM Hamiltonian for a system with {L} sites.")
print(f"Total Hilbert space dimension: {tfim_builder.total_dim}x{tfim_builder.total_dim}")

# --- Step 3: Add the nearest-neighbor S^x S^x interaction terms ---
print("\nAdding interaction terms...")
# We loop from site 0 to L-2 for open boundary conditions.
for i in range(L - 1):
    j = i + 1
    tfim_builder.add_spin_term(
        coefficient=-J,
        operators=[op.S_x, op.S_x],
        sites=[i, j]
    )
    print(f"  Added -J * S_x({i}) * S_x({j})")

# --- Step 4: Add the on-site S_z field term ---
print("\nAdding transverse field terms...")
for i in range(L):
    tfim_builder.add_spin_term(
        coefficient=-h,
        operators=[op.S_z],
        sites=[i]
    )
    print(f"  Added -h * S_z({i})")

# --- Step 5: Build the final matrix ---
H_tfim = tfim_builder.build()

print("\nSuccessfully built the Hamiltonian matrix.")
print("Shape of the final matrix:", H_tfim.shape)
print("\nFinal Hamiltonian Matrix (TFIM):")
qs.show_matrix(H_tfim)

Building a TFIM Hamiltonian for a system with 4 sites.
Total Hilbert space dimension: 16x16

Adding interaction terms...
  Added -J * S_x(0) * S_x(1)
  Added -J * S_x(1) * S_x(2)
  Added -J * S_x(2) * S_x(3)

Adding transverse field terms...
  Added -h * S_z(0)
  Added -h * S_z(1)
  Added -h * S_z(2)
  Added -h * S_z(3)

Successfully built the Hamiltonian matrix.
Shape of the final matrix: torch.Size([16, 16])

Final Hamiltonian Matrix (TFIM):


Matrix([
[ -1.0,     0,     0, -0.25,     0,     0, -0.25,     0,     0,     0,     0,     0, -0.25,     0,     0,     0],
[    0,  -0.5, -0.25,     0,     0,     0,     0, -0.25,     0,     0,     0,     0,     0, -0.25,     0,     0],
[    0, -0.25,  -0.5,     0, -0.25,     0,     0,     0,     0,     0,     0,     0,     0,     0, -0.25,     0],
[-0.25,     0,     0,     0,     0, -0.25,     0,     0,     0,     0,     0,     0,     0,     0,     0, -0.25],
[    0,     0, -0.25,     0,  -0.5,     0,     0, -0.25, -0.25,     0,     0,     0,     0,     0,     0,     0],
[    0,     0,     0, -0.25,     0,     0, -0.25,     0,     0, -0.25,     0,     0,     0,     0,     0,     0],
[-0.25,     0,     0,     0,     0, -0.25,     0,     0,     0,     0, -0.25,     0,     0,     0,     0,     0],
[    0, -0.25,     0,     0, -0.25,     0,     0,   0.5,     0,     0,     0, -0.25,     0,     0,     0,     0],
[    0,     0,     0,     0, -0.25,     0,     0,     0,  -0.5,     0,     0, -

## 4\. Building a Fermionic Model from Scratch: The Su-Schrieffer-Heeger (SSH) Model

Next, let's build a fermionic model to see how `add_fermion_term` works. The SSH Hamiltonian describes fermions on a 1D bipartite lattice:
$$H = -v \sum_{i} (c_{A,i}^\dagger c_{B,i} + \text{h.c.}) - w \sum_{i} (c_{B,i}^\dagger c_{A,i+1} + \text{h.c.})$$
We map sites as:

$A_i \to 2i$,

$B_i \to 2i+1$.

In [4]:
# --- Step 1: Define SSH model parameters ---
num_cells = 3  # Number of A-B unit cells
total_sites = 2 * num_cells
v = 1.0  # Intra-cell hopping
w = 0.5  # Inter-cell hopping

# --- Step 2: Initialize the builder ---
from quantum_simulation.hamiltonian import Hamiltonian
from quantum_simulation import operator as op

ssh_builder = Hamiltonian(lattice_length=total_sites, spin=0.5)

print(f"Building SSH model with {num_cells} cells ({total_sites} total sites).")

# --- Step 3: Add intra-cell hopping (v-terms) ---
print("\nAdding intra-cell hopping terms (v)...")
for i in range(num_cells):
    site_a = 2 * i
    site_b = 2 * i + 1
    # c_A^dag * c_B term and its Hermitian conjugate
    ssh_builder.add_fermion_term(-v, [op.c_dag_j, op.c_j], [site_a, site_b])
    ssh_builder.add_fermion_term(-v, [op.c_dag_j, op.c_j], [site_b, site_a])
    print(f"  Added hopping between sites {site_a} (A_{i}) and {site_b} (B_{i})")

# --- Step 4: Add inter-cell hopping (w-terms) ---
print("\nAdding inter-cell hopping terms (w)...")
# Loop to num_cells - 1 for open boundary conditions
for i in range(num_cells - 1):
    source_site = 2 * i + 1          # B-site of cell i
    target_site = 2 * (i + 1)        # A-site of cell i+1
    # c_B^dag * c_A term and its Hermitian conjugate
    ssh_builder.add_fermion_term(-w, [op.c_dag_j, op.c_j], [source_site, target_site])
    ssh_builder.add_fermion_term(-w, [op.c_dag_j, op.c_j], [target_site, source_site])
    print(f"  Added hopping between sites {source_site} (B_{i}) and {target_site} (A_{i+1})")

# --- Step 5: Build the final matrix ---
H_ssh = ssh_builder.build()

print("\nSuccessfully built the SSH Hamiltonian matrix.")
print("Shape of the final matrix:", H_ssh.shape)

Building SSH model with 3 cells (6 total sites).

Adding intra-cell hopping terms (v)...
  Added hopping between sites 0 (A_0) and 1 (B_0)
  Added hopping between sites 2 (A_1) and 3 (B_1)
  Added hopping between sites 4 (A_2) and 5 (B_2)

Adding inter-cell hopping terms (w)...
  Added hopping between sites 1 (B_0) and 2 (A_1)
  Added hopping between sites 3 (B_1) and 4 (A_2)

Successfully built the SSH Hamiltonian matrix.
Shape of the final matrix: torch.Size([64, 64])


## 5\. Exploring and Using Pre-Built Models

The `models` module is designed for convenience and rapid prototyping. It contains pre-built classes for many common physical models, saving you from writing the construction logic manually. A key feature is the ability to generate a symbolic, mathematical representation of the Hamiltonian using `SymPy`.

### Catalog of Available Models

#### Spin Models (1D)

  * **1D Heisenberg Model (`HeisenbergModel`)**: Describes a 1D chain of interacting spins with anisotropic couplings ($J_x, J_y, J_z$) and an external magnetic field.

      * `models.HeisenbergModel(lattice_length=10, jx=1, jy=1, jz=1, hx=0.5, pbc=True)`

  * **1D Transverse Field Ising Model (`IsingModel`)**: Features a nearest-neighbor $S^xS^x$ interaction ($-J$) and a transverse magnetic field ($-h$).

      * `models.IsingModel(lattice_length=10, j_coupling=1.0, h_field=0.5, pbc=True)`

#### Fermionic Models (1D)

  * **1D Kitaev Chain (`KitaevChain`)**: A model of spinless fermions famous for hosting Majorana zero modes. Includes terms for chemical potential, hopping, and p-wave pairing.

      * `models.KitaevChain(lattice_length=10, chemical_potential=0.5, hopping=1.0, pairing_gap=1.0, pbc=False)`

  * **Su-Schrieffer-Heeger (SSH) Model (`SSHModel`)**: A foundational model for topological physics with alternating hopping amplitudes.

      * `models.SSHModel(num_cells=5, intra_cell_hopping=1.0, inter_cell_hopping=0.5, pbc=False)`

  * **1D Hubbard Model (`HubbardModel1D`)**: Describes interacting spinful fermions with nearest-neighbor hopping ($-t$) and on-site repulsion ($U$).

      * `models.HubbardModel1D(lattice_length=5, hopping=1.0, interaction=4.0, pbc=True)`

#### 2D Lattice Models

  * **2D Heisenberg Model (`HeisenbergModel2D`)**: The 2D extension of the Heisenberg model on a square lattice.

      * `models.HeisenbergModel2D(width=4, height=4, jx=1, jy=1, jz=1, hx=0.5, pbc=True)`

  * **2D Transverse Field Ising Model (`IsingModel2D`)**: The 2D TFIM on a square lattice.

      * `models.IsingModel2D(width=4, height=4, j_coupling=1.0, h_field=0.5, pbc=True)`

  * **2D Bose-Hubbard Model (`BoseHubbardModel2D`)**: Describes soft-core bosons on a 2D square lattice with hopping, interaction, and a maximum number of bosons per site (`n_max`).

      * `models.BoseHubbardModel2D(width=3, height=3, hopping=1.0, interaction=5.0, n_max=3, pbc=True)`

  * **2D t-J Model (`TJModel2D`)**: An effective model for strongly correlated electrons with projected hopping ($-t$) and antiferromagnetic exchange ($J$).

      * `models.TJModel2D(width=3, height=3, hopping=1.0, exchange=0.4, pbc=True)`

  * **Kitaev Honeycomb Model (`KitaevHoneycombModel`)**: An exactly solvable model of a quantum spin liquid with bond-dependent Ising interactions on a honeycomb lattice.

      * `models.KitaevHoneycombModel(u_cells=2, v_cells=2, jx=1, jy=1, jz=1, pbc=True)`

### Example: Using a Pre-Built Model

Let's see how to use one of these classes in practice.

In [5]:
from quantum_simulation import models

# --- Step 1: Instantiate the model class with physical parameters ---
heisenberg_2d = models.HeisenbergModel2D(
    width=2,
    height=2,
    jx=1.0,
    jy=1.0,
    jz=1.5, # Anisotropic coupling (XXZ model)
    hx=0.3,
    hy=0.0,
    hz=0.0,
    pbc=True
)

# --- Step 2: Build the Hamiltonian matrix ---
H_heisenberg_2d = heisenberg_2d.build()

print("Built 2x2 Heisenberg Hamiltonian.")
print("Matrix shape:", H_heisenberg_2d.shape)

# --- Step 3: Display the symbolic Hamiltonian ---
heisenberg_2d.display_hamiltonian()

Built 2x2 Heisenberg Hamiltonian.
Matrix shape: torch.Size([16, 16])
Symbolic Hamiltonian:


\sum_{\langle i,j \rangle}(Jₓ⋅S_i__x⋅S_j__x + J_y⋅S_i__y⋅S_j__y + J_z⋅S_i__z⋅S ↪

↪ _j__z) - \sum_{i}(S_i__x⋅hₓ + S_i__y⋅h_y + S_i__z⋅h_z)

## 6\. Special Case: The `add_prebuilt_term` Method

If you have a term that is difficult to construct or already calculated, the `add_prebuilt_term` method allows you to add a raw matrix directly. For example, we can add a constant energy shift, $C \times I$, to the system.

In [6]:
from quantum_simulation.hamiltonian import Hamiltonian
from quantum_simulation import operator as op
import quantum_simulation as qs

# We'll re-initialize a builder for this example
L = 4
h = 0.5
builder_prebuilt = Hamiltonian(lattice_length=L, spin=0.5)

# Add a normal term
builder_prebuilt.add_spin_term(coefficient=-h, operators=[op.S_z], sites=[0])

# Now, create and add a pre-built identity matrix term
energy_shift_constant = 10.0
identity_matrix = torch.eye(builder_prebuilt.total_dim)
prebuilt_term_matrix = energy_shift_constant * identity_matrix

builder_prebuilt.add_prebuilt_term(prebuilt_term_matrix)
print(f"Added a pre-built term corresponding to an energy shift of {energy_shift_constant}.")

# Build the final Hamiltonian
H_with_shift = builder_prebuilt.build()

print("\nHamiltonian with only H_z(0) term and energy shift:")
qs.show_matrix(H_with_shift)

Added a pre-built term corresponding to an energy shift of 10.0.

Hamiltonian with only H_z(0) term and energy shift:


⎡9.75   0     0     0     0     0     0     0      0      0      0      0      ↪
⎢                                                                              ↪
⎢ 0    9.75   0     0     0     0     0     0      0      0      0      0      ↪
⎢                                                                              ↪
⎢ 0     0    9.75   0     0     0     0     0      0      0      0      0      ↪
⎢                                                                              ↪
⎢ 0     0     0    9.75   0     0     0     0      0      0      0      0      ↪
⎢                                                                              ↪
⎢ 0     0     0     0    9.75   0     0     0      0      0      0      0      ↪
⎢                                                                              ↪
⎢ 0     0     0     0     0    9.75   0     0      0      0      0      0      ↪
⎢                                                                              ↪
⎢ 0     0     0     0     0 

## 7\. Conclusion

Congratulations\! You have now learned how to use the `quantum_simulation` toolkit to construct Hamiltonians for arbitrary lattice models.

You have learned to:

  * **Build from scratch:** Use the `Hamiltonian` class with `add_spin_term` and `add_fermion_term` for full, low-level control.
  * **Use convenience classes:** Leverage the `models` module to quickly generate Hamiltonians for common physical systems and to visualize their mathematical formulas.
  * **Handle special cases:** Inject pre-calculated matrices directly into the builder when needed.

This flexible, multi-layered design allows for both rapid prototyping of standard models and detailed construction of novel, complex Hamiltonians. Happy simulating\!